In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Set the path to your Chrome driver executable
driver_path ='C:/Users/ahmad/Downloads/chromedriver-win64/chromedriver.exe'

# Initialize the Chrome service and driver
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(service=service, options=options)

# URL to scrape
url = 'https://www.youtube.com/@ZahidAliTahir/featured'

# Load the webpage and maximize the window
driver.get(url)
driver.maximize_window()
time.sleep(2)

# Click on the "Videos" tab
videos_button = driver.find_element(By.XPATH, '//*[@id="tabsContent"]/yt-tab-group-shape/div[1]/yt-tab-shape[2]/div[1]')
videos_button.click()

# Scroll down to load more videos
for _ in range(5):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)

# Get video links
videos = driver.find_elements(By.CLASS_NAME, "style-scope ytd-rich-grid-media")
link_array = []
for video in videos:
    link = video.find_element(By.XPATH, './/*[@id="video-title-link"]').get_attribute('href')
    link_array.append(link)

# Close the WebDriver
driver.quit()

# Scrape comments for each video and append to CSV
for link in link_array:
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(link)
    time.sleep(2)

    # Scroll down to load more comments
    for _ in range(5):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)

    comments = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    arr = []
    for comment in comments:
        arr.append(comment.text)

    # Create a DataFrame for the comments
    df = pd.DataFrame({'Comments': arr})

    # Append comments to the CSV file
    df.to_csv('youtube_comments.csv', mode='a', header=False, index=False)

    # Close the video tab
    driver.close()
